In [1]:
import os
import time
import h5py
import json
import keras
import numpy as np
import pandas as pd
import seaborn as sb
import tensorflow as tf
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
from PIL import Image
from keras.optimizers import Adam
from keras.utils import plot_model
from matplotlib.image import imread
from keras.preprocessing import image
from mixup_generator import MixupGenerator
from random_eraser import get_random_eraser 
from keras.models import Sequential, load_model
from keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3, EfficientNetB4, EfficientNetB7
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization

# Dados
## Explorando dados

In [2]:
# Importando csv do dataset
data = pd.read_csv('dados/train.csv', sep = ',')
data.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [3]:
# Lendo json e crinado mapping para para classe
with open('dados/label_num_to_disease_map.json') as f:
    mapping = json.loads(f.read())
    mapping = {int(k): v for k, v in mapping.items()}

mapping

{0: 'Cassava Bacterial Blight (CBB)',
 1: 'Cassava Brown Streak Disease (CBSD)',
 2: 'Cassava Green Mottle (CGM)',
 3: 'Cassava Mosaic Disease (CMD)',
 4: 'Healthy'}

In [4]:
# Adicionando label_name ao conjunto de dados, e ondenando colunas
data['label_name'] = data['label'].map(mapping)
data = data[['image_id', 'label_name', 'label']]

In [5]:
data.head()

,image_id,label_name,label
0,1000015157.jpg,Cassava Bacterial Blight (CBB),0
1,1000201771.jpg,Cassava Mosaic Disease (CMD),3
2,100042118.jpg,Cassava Brown Streak Disease (CBSD),1
3,1000723321.jpg,Cassava Brown Streak Disease (CBSD),1
4,1000812911.jpg,Cassava Mosaic Disease (CMD),3


In [6]:
# Verificando frequencia
data.label_name.value_counts()

Cassava Mosaic Disease (CMD)           13158
Healthy                                 2577
Cassava Green Mottle (CGM)              2386
Cassava Brown Streak Disease (CBSD)     2189
Cassava Bacterial Blight (CBB)          1087
Name: label_name, dtype: int64

In [7]:
# Verificando proporção 
data.label_name.value_counts(normalize = True) * 100

Cassava Mosaic Disease (CMD)           61.494602
Healthy                                12.043744
Cassava Green Mottle (CGM)             11.151096
Cassava Brown Streak Disease (CBSD)    10.230406
Cassava Bacterial Blight (CBB)          5.080151
Name: label_name, dtype: float64

# Preparando os dados

## HoldOut
Separando dados de treino e teste

In [5]:
# Definindo paths de treino e teste
TRAIN_PATH = 'dados/train_images/'
TEST_PATH = 'dados/test_images/'
IMAGE_TEST = 'dados/test_images/2216849948.jpg'

In [6]:
# Criando amostra para treino e teste estrafiticada
train, test = train_test_split(data, test_size = 0.10, shuffle = True, random_state = 0, stratify = data['label_name'])

In [10]:
# Verificando frequencia do conjunto de trian
train.label_name.value_counts()

Cassava Mosaic Disease (CMD)           11842
Healthy                                 2319
Cassava Green Mottle (CGM)              2148
Cassava Brown Streak Disease (CBSD)     1970
Cassava Bacterial Blight (CBB)           978
Name: label_name, dtype: int64

In [11]:
# Verificando frequencia do conjunto de test
test.label_name.value_counts()

Cassava Mosaic Disease (CMD)           1316
Healthy                                 258
Cassava Green Mottle (CGM)              238
Cassava Brown Streak Disease (CBSD)     219
Cassava Bacterial Blight (CBB)          109
Name: label_name, dtype: int64

In [13]:
train.head()

,image_id,label_name,label
10866,2937152803.jpg,Cassava Mosaic Disease (CMD),3
11253,3009725992.jpg,Cassava Mosaic Disease (CMD),3
5623,1999385559.jpg,Cassava Mosaic Disease (CMD),3
14958,3687069352.jpg,Cassava Brown Streak Disease (CBSD),1
20700,858590964.jpg,Cassava Mosaic Disease (CMD),3


In [14]:
test.head()

,image_id,label_name,label
14906,3674975223.jpg,Cassava Mosaic Disease (CMD),3
3794,1674922822.jpg,Cassava Mosaic Disease (CMD),3
19818,698931163.jpg,Cassava Mosaic Disease (CMD),3
16499,3966311153.jpg,Cassava Mosaic Disease (CMD),3
14827,3658819051.jpg,Cassava Mosaic Disease (CMD),3


In [7]:
mixup_x_train = []
mixup_y_train = train.label.values
mixup_y_train = keras.utils.to_categorical(mixup_y_train, 5)

def image_mixup_train (x):
    img = image.load_img(TRAIN_PATH + x, target_size = (456, 456))
    img = image.img_to_array(img)
    mixup_x_train.append(img)


mixup_x_test = []
mixup_y_test = test.label.values
mixup_y_test = keras.utils.to_categorical(mixup_y_test, 5)

def image_mixup_test (x):
    img = image.load_img(TRAIN_PATH + x, target_size = (456, 456))
    img = image.img_to_array(img)
    mixup_x_test.append(img)

In [8]:
test['image_id'].map(image_mixup_test)

14906    None
3794     None
19818    None
16499    None
14827    None
         ... 
5886     None
6045     None
5052     None
1156     None
10114    None
Name: image_id, Length: 2140, dtype: object

In [9]:
with h5py.File('dados/mixup_test.h5', 'w') as hf:
    hf.create_dataset('mixup_x_test', dtype = 'uint8', data = mixup_x_test)
    hf.create_dataset('mixup_y_test', dtype = 'uint8', data = mixup_y_test)

In [14]:
train['image_id'].map(image_mixup_train)

In [ ]:
with h5py.File('dados/mixup_train.h5', 'w') as hf:
    hf.create_dataset('mixup_x_train', dtype = 'uint8', data = mixup_x_train)
    hf.create_dataset('mixup_y_train', dtype = 'uint8', data = mixup_y_train)

## Tratamento das imagens
Nossas imagens não estão no formato adequado para usarmos em uma rede neural, usaremos a classe **ImageDataGenerator** com o method **flow_from_dataframe** do keras. Ele irar percorrer nosso cvs e diretório de dados, carregando os dados de imagem e retornar a entrada (matrizes de pixels) e a saída (número inteiro de classe).

## Data Augumentation

É uma técnica para gerar novos exemplares de dados de treinamento a fim de aumentar a generalidade do modelo. Técnicas de aumento de dados, como corte, preenchimento, redimensionamento e rotação, são comumente usadas para treinar grandes redes neurais.

A ideia de manipular imagens decorre de uma premissa básica das atividades que envolvem Deep Learning e a Aprendizagem de Máquina, que é a representatividade da amostra de dados utilizada para Treinar um modelo. Quanto maior for a amostra e mais representativos forem os dados utilizados na etapa de treinamento, melhor será o desempenho do modelo ao classificar novos dados. Assim, ao rotacionar as imagens e redimensiona-las, aumentamos nosso espaço amostral gerando uma maior variabilidade de dados, fazendo com que nosso modelo aprenda características apresentadas sob outra "forma".

### Definindo parâmetros para tratamnetos das imagens
Abaixo estamos aumentando nossos dados ao gerar novas imagens a partir das imagens já existentes. Cada parâmetro passado a nossa classe **ImageDataGenerator** altera características das imagens existentes, como zoom, largura, rotação, escala e etc. Um exemplo pode ser visto nas imagens abaixo.

In [12]:
train_generator = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
    #preprocessing_function = get_random_eraser(v_l = 0, v_h = 255),
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
    #rescale = 1/225
)

test_generator = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input
)

In [ ]:
# Plotando imagens com aumento usando ImageDataGenerator do keras
image_list = os.listdir(TRAIN_PATH)
count = 0

plt.figure(figsize = (20, 10))
for i in image_list:
    img = imread(TRAIN_PATH + i)
    plt.subplot(2, 3, count + 1)
    plt.suptitle("Data Augumentation", fontsize = 30)
    plt.imshow(train_generator.random_transform(img))
    plt.title(f"{i} \n {data.loc[data.image_id == i, 'label_name'].values}")
    plt.axis('off')
    count += 1
    if(count == 6):
        break

In [ ]:
train_set = train_generator.flow_from_dataframe(
    dataframe = train,
    directory = TRAIN_PATH,
    seed = 42,
    x_col = 'image_id',
    y_col = 'label_name',
    target_size = (456, 456),
    class_mode = 'categorical',
    interpolation = 'nearest',
    shuffle = True,
    batch_size = 15
)

test_set = test_generator.flow_from_dataframe(
    dataframe = test,
    directory= TRAIN_PATH,
    seed = 42,
    x_col = 'image_id',
    y_col = 'label_name',
    target_size = (456, 456),
    class_mode = 'categorical',
    interpolation = 'nearest',
    shuffle = True,
    batch_size = 15
)

## MixUp

In [ ]:
#
mixup_gen = MixupGenerator(mixup_x_train, mixup_y_train, datagen = train_generator, alpha = 0.5)()
x, y = next(mixup_gen)

plt.figure(figsize = (20, 10))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(x[i].astype(np.uint8), interpolation = "nearest")
    plt.axis('off')

# Modelos

## Trabalhando hiperparâmetros da rede neural

In [ ]:
# Definindo loss function
loss = CategoricalCrossentropy(from_logits = False, label_smoothing = 0.0001, name = 'categorical_crossentropy')
# Pare de treinar quando o val_loss parar de diminuir por 3 épocas
es = EarlyStopping(monitor = 'val_loss', patience = 3, verbose = 1, mode = 'min', restore_best_weights = True)
# Salve o modelo com o mínimo de perda de validação
checkpoint = ModelCheckpoint("CassavaLeafDisease.h5", monitor = 'val_loss', save_best_only = True, mode = 'min')
# Reduza a taxa de aprendizagem quando a aprendizagem estagnar
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 2, min_lr = 1e-6, mode = 'min', verbose = 1)

## EfficientNetB3

In [ ]:
# Criando modelo de rede neural
modelo = Sequential()
modelo.add(EfficientNetB3(input_shape = (456, 456, 3), include_top = False, weights = 'imagenet', drop_connect_rate = 0.6))
modelo.add(GlobalAveragePooling2D())
modelo.add(Flatten())
modelo.add(Dense(256, activation = 'relu', bias_regularizer = tf.keras.regularizers.L1L2(l1 = 0.01, l2 = 0.001)))
modelo.add(Dropout(0.5))
modelo.add(Dense(5, activation = 'softmax'))

In [ ]:
# Bloqueando pesos da EfficientNetB3
modelo.layers[0].trainable = True
set_trainable = False

# Desbloqueando pesos da layer block1a_se_excite em diante
for layer in modelo.layers[0].layers:
    if layer.name == 'block1a_se_excite':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# Compilando rede neural
modelo.compile(optimizer = Adam(lr = 0.001, decay = 0.0001, clipvalue = 0.5), loss = loss, metrics = ['categorical_accuracy'])

In [ ]:
# Informações de parâmetros do modelo EfficientNetB3
modelo.layers[0].summary()

In [ ]:
# Informações de parâmetros do modelo 
modelo.summary()

In [ ]:
# Plotando fluxograma do modelo EfficientNetB3
plot_model(modelo.layers[0], to_file = 'img/efficientnetb3.png',  show_shapes = True)

In [ ]:
# Plotando fluxograma do modelo
plot_model(modelo, to_file = 'img/modelo.png',  show_shapes = True)

## Usando Data Augumentation com ImageDataGenerator

In [ ]:
# Definindo parâmetros para treino e teste
STEP_SIZE_TRAIN = train_set.n // train_set.batch_size
STEP_SIZE_TEST = test_set.n // test_set.batch_size

inicio = time.time()

# Treinamento e teste do modelo
results = modelo.fit(
        train_set, 
        validation_data = test_set, 
        epochs = 15, 
        batch_size = 15, 
        steps_per_epoch = STEP_SIZE_TRAIN,
        validation_steps = STEP_SIZE_TEST, 
        callbacks = [es, checkpoint, reduce_lr]
)

print()
fim = time.time()
print(fim - inicio)

# Salvando modelo
# modelo.save('CassavaLeafDisease')

## Usando Data Augumentation com ImageDataGenerator, MixUp e Random Erasing

# Submission

In [ ]:
# Carregando modelo
leaf_model = load_model('CassavaLeafDisease.h5')

In [ ]:
# Realizando predição com imagem teste 
test_images = os.listdir(TEST_PATH)
predictions = []

for image in test_images:
    img = Image.open(TEST_PATH + image)
    img = img.resize((456, 456), resample = Image.NEAREST) 
    img = np.expand_dims(img, axis = 0)
    # Realizando a predição, argmax retorna o indice da maior probabilidade da lista
    predictions.extend(leaf_model.predict(img).argmax(axis = 1))
    predictions

In [ ]:
# Realizando predição com imagem teste 
test_images = os.listdir(TEST_PATH)
predictions = []

for image in test_images:
    teste4 = image.load_img(TEST_PATH + image, target_size = (456, 456))
    teste4 = np.expand_dims(teste4, axis = 0)
    # Realizando a predição, argmax retorna o indice da maior probabilidade da lista
    predictions.extend(leaf_model.predict(teste4).argmax(axis = 1))
    predictions

In [ ]:
# Gerando arquivo submission.csv
submission = pd.DataFrame({'image_id': test_images, 'label': predictions})
submission.to_csv('submission.csv', index = False)
display(submission)